# SRM Forward Motor Simulation
고체로켓 모터(SRM) Forward 시뮬레이션 - 그레인 형상별 추력 곡선 생성

## 개선사항
1. **FFT 가속 (Phase-Field)**: Laplacian을 FFT로 계산 (O(N³) → O(N³logN))
2. **적응 타임스텝 (CFL)**: 안정성 기반 자동 dt 조정
3. **Semi-Implicit 스킴**: Allen-Cahn 표준 방법 (더 안정적)
4. **그래프 개선**: 마커/라인 스타일 강조
5. **성능 분석**: Impulse, Impulse Density, Burn Rate 통계
6. **3D 시각화**: 슬라이스 및 등밀도 표면 (isosurface)
7. **민감도 분석**: 파라미터 변화에 따른 F(t) 응답

In [ ]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("[Init] Libraries loaded successfully")

## 1. 물리 파라미터 및 격자 설정

In [ ]:
# ===================== 물리 파라미터 =====================
gamma = 1.22  # 비열비
R = 355.0  # 가스상수 [J/kgK]
Tc = 3200.0  # 연소 온도 [K]
rho_p = 1700.0  # 프로펠란트 밀도 [kg/m³]
a = 5.0e-5  # 연소율 상수 [m/(s·Pa^n)]
n = 0.35  # 연소율 지수
pa = 101325.0  # 대기압 [Pa]
At = 3.0e-4  # 목 면적 [m²]

# ===================== 격자 파라미터 =====================
L = 0.20  # 길이 [m]
R_case = 0.10  # 케이스 반경 [m]
N = 32  # 격자 해상도 (32^3)
dx = max(L, 2*R_case) / N  # 격자 간격
eps = 4.0 * dx  # 위상장 정규화 파라미터

print(f"[Grid] N={N}, dx={dx:.6f}m, eps={eps:.6f}m")
print(f"[Physical] γ={gamma}, Tc={Tc}K, ρ_p={rho_p}kg/m³, r=a*Pc^n (a={a}, n={n})")

In [ ]:
# 격자 생성
z_lin = np.linspace(-L/2, L/2, N)
y_lin = np.linspace(-R_case, R_case, N)
x_lin = np.linspace(-R_case, R_case, N)
Z, Y, X = np.meshgrid(z_lin, y_lin, x_lin, indexing='ij')
radial = np.sqrt(X**2 + Y**2)
angle = np.arctan2(Y, X)

# FFT 파라미터 (개선사항 #1)
kx = 2 * np.pi * np.fft.fftfreq(N, d=dx)
k2 = kx[None, None, :]**2 + kx[None, :, None]**2 + kx[:, None, None]**2
print(f"[FFT] k² computed for Laplacian acceleration")

In [ ]:
# ===================== 시간 그리드 =====================
dt_base = 0.001  # 기본 타임스텝
t_end = 3.5  # 시뮬 종료시간
cfl_default = 0.2  # adaptive CFL 계수 (adaptive_timestep 기본값과 일치)
r_dot_max = a * (1.0e8)**n  # Pc 클램프 상한 기준 최대 연소율
dt_cfl_min = cfl_default * dx / max(r_dot_max, 1e-8)
dt_min = min(dt_base, dt_cfl_min)
max_steps = int(np.ceil(t_end / dt_min)) + 1

# ===================== 출력 디렉토리 =====================
OUT_DIR = "./out_thrust_improved"
os.makedirs(OUT_DIR, exist_ok=True)

print(f"[Time] dt_base={dt_base}, dt_min={dt_min:.6f}, t_end={t_end}, max_steps={max_steps}")
print(f"[Output] {OUT_DIR}")


## 2. 핵심 함수 정의

In [ ]:
def mdot_choked(Pc):
    """Choked flow mass flow rate (질식 유동)"""
    C = math.sqrt(gamma / (R * Tc)) * (2.0 / (gamma + 1.0)) ** ((gamma + 1.0) / (2.0 * (gamma - 1.0)))
    return At * np.maximum(Pc, 0.0) * C

def exhaust_velocity(Pc):
    """등엔트로피 배기속도"""
    Pc_eff = np.clip(Pc, pa * 1.1, 1e8)
    return np.sqrt(2.0 * gamma * R * Tc / (gamma - 1.0) * (1.0 - (pa / Pc_eff)**((gamma - 1.0)/gamma)))

def double_well(phi):
    """Double well potential W(φ) = φ²(1-φ)²"""
    return phi**2 * (1.0 - phi)**2

def grad_norm_squared(phi):
    """|∇φ|² (중앙차분)"""
    gx = (np.roll(phi, -1, axis=2) - np.roll(phi, 1, axis=2)) / (2 * dx)
    gy = (np.roll(phi, -1, axis=1) - np.roll(phi, 1, axis=1)) / (2 * dx)
    gz = (np.roll(phi, -1, axis=0) - np.roll(phi, 1, axis=0)) / (2 * dx)
    return gx**2 + gy**2 + gz**2 + 1e-12

def interface_measure(phi):
    """표면적 계산: ∫(ε|∇φ|² + W/ε)dV"""
    grad_sq = grad_norm_squared(phi)
    return np.sum(eps * grad_sq + double_well(phi) / eps) * dx**3

def propellant_volume(phi):
    """프로펠란트 부피"""
    return np.sum(np.clip(phi, 0.0, 1.0)) * dx**3

def total_volume():
    """케이스 부피"""
    return math.pi * R_case**2 * L

def loading_fraction(phi):
    """로딩 분율"""
    return propellant_volume(phi) / total_volume()

print("[Functions] Core physics functions defined")

In [ ]:
def adaptive_timestep(phi, Pc, r_dot, cfl=cfl_default):
    """개선사항 #2: CFL 기반 적응 타임스텝"""
    # 특성 속도: burning velocity
    v_char = r_dot + 1e-10
    # CFL: dt <= cfl * dx / v_char
    dt_cfl = cfl * dx / np.maximum(v_char, 1e-8)
    return min(dt_base, np.min(dt_cfl))

def laplacian_fft(phi):
    """개선사항 #1: FFT 기반 Laplacian (O(N³logN), 주기 경계)"""
    phi_fft = np.fft.fftn(phi)
    laplacian_fft = -k2 * phi_fft
    return np.fft.ifftn(laplacian_fft).real

def semi_implicit_step(phi, W_prime, r_dot, dt):
    """개선사항 #3: Semi-implicit Allen-Cahn 스텝
    ∂φ/∂t = (1/ε)W'(φ) - (r/ε)²Δφ
    """
    # 우변: explicit nonlinearity + implicit Laplacian
    rhs = phi + (dt / eps) * W_prime
    
    # Denominator: 1 + dt * (r/eps)² * k² (주기 BC용)
    denom = 1.0 + dt * (r_dot / eps)**2 * k2
    denom = np.clip(denom, 1e-6, None)
    
    # FFT 풀이
    rhs_fft = np.fft.fftn(rhs)
    phi_fft = rhs_fft / denom
    phi_new = np.fft.ifftn(phi_fft).real
    
    return phi_new

print("[Numerics] Improved time-stepping functions defined")

## 3. 그레인 생성기

In [ ]:
def create_cylindrical_grain(r0=0.055):
    """원통형 그레인 (기본)"""
    phi = np.zeros((N, N, N))
    phi[radial <= r0] = 1.0
    return phi

def create_star_grain(r_outer=0.055, num_points=6):
    """별형 그레인 (6점)"""
    r_envelope = r_outer * (0.5 + 0.5 * np.cos(num_points * angle))
    phi = np.zeros((N, N, N))
    phi[radial <= r_envelope] = 1.0
    return phi


def create_finocyl_approx(r_outer=0.055, r_inner=0.025):
    """Finocyl 근사 (6 slot)"""
    phi = np.zeros((N, N, N))
    inner_mask = radial <= r_inner
    outer_mask = (radial > r_inner) & (radial <= r_outer)
    angle_slot = (angle % (np.pi / 3)) - np.pi / 6
    slot_presence = np.abs(angle_slot) > (np.pi / 8)
    phi[inner_mask | (outer_mask & slot_presence)] = 1.0
    return phi


def create_rod_and_tube(r_outer=0.055, r_inner=0.010):
    """개선사항 추가: Rod-and-Tube (환형)"""
    phi = np.zeros((N, N, N))
    phi[(radial >= r_inner) & (radial <= r_outer)] = 1.0
    return phi

print("[Grains] 4 grain types created")

## 4. Forward Motor Simulation (개선 버전)

In [ ]:
def forward_motor_simulation(phi_init, grain_name="grain", use_semi_implicit=True):
    """
    Forward 모터 시뮬레이션 - 개선된 버전

    Args:
        phi_init: 초기 phi 필드 (N, N, N)
        grain_name: 그레인 이름
        use_semi_implicit: semi-implicit 스킴 사용 여부

    Returns:
        dict: 히스토리
    """
    phi = phi_init.copy().astype(np.float64)
    Pc = 2.0e6  # 초기 압력

    history = {
        "t": [],
        "Pc": [],
        "F": [],
        "Ab": [],
        "vol": [],
        "load": [],
        "r_dot": [],
        "dt": [],
        "Impulse": 0.0  # Total impulse
    }

    print(f"\n[Forward] {grain_name} (semi_implicit={use_semi_implicit})")
    print(f"  Initial: Load={loading_fraction(phi):.3f}, Ab={interface_measure(phi):.4e}")

    t = 0.0
    step = 0
    while t < t_end and step < max_steps:
        # 표면적
        Ab = interface_measure(phi)

        # 연소율
        Pc_clamped = np.clip(Pc, 1.0, 1e8)
        r_dot = a * Pc_clamped**n
        dt = adaptive_timestep(phi, Pc_clamped, r_dot)

        # 질량 흐름
        mdot_gen = rho_p * Ab * r_dot
        mdot_noz = mdot_choked(Pc_clamped)

        # 기체 부피
        V_gas = np.sum(1.0 - np.clip(phi, 0.0, 1.0)) * dx**3
        V_gas = np.clip(V_gas, 1e-6, total_volume() * 1.1)

        # 압력 변화
        dPdt = (mdot_gen - mdot_noz) * R * Tc / V_gas
        Pc = Pc + dPdt * dt
        Pc = np.clip(Pc, 0.0, 1e8)

        # 추력
        ve = exhaust_velocity(Pc)
        F = mdot_noz * ve
        history["Impulse"] += F * dt  # 누적 임펄스

        # phi 진화
        if use_semi_implicit:
            W_prime = 2.0 * phi * (1.0 - phi) * (1.0 - 2.0 * phi)
            phi = semi_implicit_step(phi, W_prime, r_dot, dt)
        else:
            # Explicit Euler (기존 방식)
            W_prime = 2.0 * phi * (1.0 - phi) * (1.0 - 2.0 * phi)
            laplacian = (np.roll(phi, 1, axis=0) + np.roll(phi, -1, axis=0) +
                         np.roll(phi, 1, axis=1) + np.roll(phi, -1, axis=1) +
                         np.roll(phi, 1, axis=2) + np.roll(phi, -1, axis=2) - 6*phi) / (dx**2)
            phi = phi + dt * ((W_prime / eps) - (r_dot / eps)**2 * laplacian)

        phi = np.clip(phi, -0.2, 1.2)
        # 시그모이드 정규화
        phi = 1.0 / (1.0 + np.exp(-20.0 * (phi - 0.5)))
        phi = np.clip(phi, 0.0, 1.0)

        # 히스토리 저장
        history["t"].append(t)
        history["Pc"].append(Pc)
        history["F"].append(F)
        history["Ab"].append(Ab)
        history["vol"].append(propellant_volume(phi))
        history["load"].append(loading_fraction(phi))
        history["r_dot"].append(r_dot)
        history["dt"].append(dt)

        t += dt
        step += 1

        if step % 500 == 0:
            load_val = history["load"][-1]
            print(f"  Step {step:4d}: t={t:.3f}s, Pc={Pc/1e6:.2f}MPa, F={F/1e3:.2f}kN, Load={load_val:.3f}")


    if step >= max_steps and t < t_end:
        print(f"  [Warn] Reached max_steps={max_steps} before t_end={t_end}s (t={t:.3f}s).")

    # numpy array로 변환
    for key in ["t", "Pc", "F", "Ab", "vol", "load", "r_dot", "dt"]:
        history[key] = np.array(history[key])

    # 성능 지표 (개선사항 #5)
    print(f"  Final: Pc={history['Pc'][-1]/1e6:.2f}MPa, F={history['F'][-1]/1e3:.2f}kN")
    print(f"  Total Impulse: {history['Impulse']/1e3:.2f} kN·s")

    return history


## 5. 시뮬레이션 실행

In [ ]:
# 그레인 생성
print("=== Grain Creation ===")
phi_cylinder = create_cylindrical_grain(r0=0.055)
phi_star = create_star_grain(r_outer=0.055, num_points=6)
phi_finocyl = create_finocyl_approx(r_outer=0.055, r_inner=0.025)
phi_rod_tube = create_rod_and_tube(r_outer=0.055, r_inner=0.010)  # 새 그레인

grains = {
    "Cylinder": phi_cylinder,
    "Star": phi_star,
    "Finocyl": phi_finocyl,
    "Rod-Tube": phi_rod_tube
}

for name, phi in grains.items():
    print(f"{name:12s}: Load={loading_fraction(phi):.3f}, Ab={interface_measure(phi):.4e}")

In [ ]:
# Forward 시뮬레이션 실행
print("\n=== Forward Simulation ===")
histories = {}
for name, phi in grains.items():
    histories[name] = forward_motor_simulation(phi, grain_name=name, use_semi_implicit=True)

## 6. 시각화 및 분석

In [ ]:
# 색상 정의
colors = {"Cylinder": "blue", "Star": "red", "Finocyl": "green", "Rod-Tube": "purple"}

# 4 패널 플롯
fig, axs = plt.subplots(2, 2, figsize=(14, 10))

# Pc(t)
for name, hist in histories.items():
    axs[0, 0].plot(hist["t"], hist["Pc"]/1e6, color=colors[name], label=name, linewidth=2.5)
axs[0, 0].set_xlabel('Time [s]', fontsize=11)
axs[0, 0].set_ylabel('Chamber Pressure [MPa]', fontsize=11)
axs[0, 0].legend(fontsize=10)
axs[0, 0].grid(True, alpha=0.3)
axs[0, 0].set_title('Chamber Pressure vs Time', fontsize=12, fontweight='bold')

# F(t)
for name, hist in histories.items():
    axs[0, 1].plot(hist["t"], hist["F"]/1e3, color=colors[name], label=name, linewidth=2.5)
axs[0, 1].set_xlabel('Time [s]', fontsize=11)
axs[0, 1].set_ylabel('Thrust [kN]', fontsize=11)
axs[0, 1].legend(fontsize=10)
axs[0, 1].grid(True, alpha=0.3)
axs[0, 1].set_title('Thrust vs Time', fontsize=12, fontweight='bold')

# Ab(t)
for name, hist in histories.items():
    axs[1, 0].plot(hist["t"], hist["Ab"], color=colors[name], label=name, linewidth=2.5)
axs[1, 0].set_xlabel('Time [s]', fontsize=11)
axs[1, 0].set_ylabel('Burning Area [m²]', fontsize=11)
axs[1, 0].legend(fontsize=10)
axs[1, 0].grid(True, alpha=0.3)
axs[1, 0].set_title('Burning Area vs Time', fontsize=12, fontweight='bold')

# Loading(t)
for name, hist in histories.items():
    axs[1, 1].plot(hist["t"], hist["load"], color=colors[name], label=name, linewidth=2.5)
axs[1, 1].set_xlabel('Time [s]', fontsize=11)
axs[1, 1].set_ylabel('Loading Fraction', fontsize=11)
axs[1, 1].legend(fontsize=10)
axs[1, 1].grid(True, alpha=0.3)
axs[1, 1].set_title('Loading Fraction vs Time', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "grain_comparison.png"), dpi=150, bbox_inches='tight')
plt.show()
print(f"[Save] grain_comparison.png")

In [ ]:
# 추력 강조 플롯 (개선사항 #4: 마커 강조)
fig, ax = plt.subplots(figsize=(12, 7))

for name, hist in histories.items():
    ax.plot(hist["t"], hist["F"]/1e3, color=colors[name], label=name, 
            linewidth=3, marker='o', markersize=2, markevery=150)

ax.set_xlabel('Time [s]', fontsize=12, fontweight='bold')
ax.set_ylabel('Thrust [kN]', fontsize=12, fontweight='bold')
ax.legend(fontsize=11, loc='upper right')
ax.grid(True, alpha=0.4, linestyle='--')
ax.set_title('SRM Thrust Profile - Grain Shape Comparison', fontsize=13, fontweight='bold')
ax.set_xlim(0, 3.5)

plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "thrust_comparison.png"), dpi=150, bbox_inches='tight')
plt.show()
print(f"[Save] thrust_comparison.png")

## 7. 성능 분석 (개선사항 #5)

In [ ]:
print("\n" + "="*70)
print(" " * 15 + "PERFORMANCE ANALYSIS")
print("="*70)

for name, hist in histories.items():
    Pc_avg = np.mean(hist["Pc"])
    Pc_max = np.max(hist["Pc"])
    Pc_min = np.min(hist["Pc"])
    
    F_avg = np.mean(hist["F"])
    F_peak = np.max(hist["F"])
    F_min = np.min(hist["F"])
    
    impulse = hist["Impulse"]
    impulse_density = impulse / propellant_volume(grains[name])  # impulse density proxy
    
    load_initial = hist["load"][0]
    load_final = hist["load"][-1]
    
    print(f"\n[{name}]")
    print(f"  Pressure:")
    print(f"    Avg={Pc_avg/1e6:6.2f}MPa, Peak={Pc_max/1e6:6.2f}MPa, Min={Pc_min/1e6:6.2f}MPa")
    print(f"  Thrust:")
    print(f"    Avg={F_avg/1e3:6.2f}kN, Peak={F_peak/1e3:6.2f}kN, Min={F_min/1e3:6.2f}kN")
    print(f"  Impulse:")
    print(f"    Total={impulse/1e3:7.2f}kN·s, ImpulseDensity={impulse_density:7.2f}(J/m³)")
    print(f"  Loading:")
    print(f"    Initial={load_initial:.3f}, Final={load_final:.3f}, Burned={load_initial-load_final:.3f}")

print("\n" + "="*70)

## 8. 민감도 분석 (개선사항 #7)

In [ ]:
# 연소율 상수 a의 변화에 따른 영향
print("\n=== Sensitivity Analysis: Burn Rate Parameter 'a' ===")

a_values = [a*0.8, a, a*1.2]  # -20%, nominal, +20%
sens_histories = {}

# 원통 그레인만 사용 (기준)
for a_test in a_values:
    # 새로운 시뮬 함수 (a 파라미터 포함)
    phi = create_cylindrical_grain(r0=0.055)
    Pc = 2.0e6
    hist = {"t": [], "F": []}
    
    t = 0.0
    step = 0
    while t < t_end and step < max_steps:
        Ab = interface_measure(phi)
        Pc_clamped = np.clip(Pc, 1.0, 1e8)
        r_dot = a_test * Pc_clamped**n
        dt = adaptive_timestep(phi, Pc_clamped, r_dot)
        mdot_gen = rho_p * Ab * r_dot
        mdot_noz = mdot_choked(Pc_clamped)
        V_gas = np.sum(1.0 - np.clip(phi, 0.0, 1.0)) * dx**3
        V_gas = np.clip(V_gas, 1e-6, total_volume() * 1.1)
        dPdt = (mdot_gen - mdot_noz) * R * Tc / V_gas
        Pc = Pc + dPdt * dt
        Pc = np.clip(Pc, 0.0, 1e8)
        ve = exhaust_velocity(Pc)
        F = mdot_noz * ve
        
        W_prime = 2.0 * phi * (1.0 - phi) * (1.0 - 2.0 * phi)
        laplacian = (np.roll(phi, 1, axis=0) + np.roll(phi, -1, axis=0) +
                     np.roll(phi, 1, axis=1) + np.roll(phi, -1, axis=1) +
                     np.roll(phi, 1, axis=2) + np.roll(phi, -1, axis=2) - 6*phi) / (dx**2)
        phi = phi + dt * ((W_prime / eps) - (r_dot / eps)**2 * laplacian)
        phi = np.clip(phi, 0.0, 1.0)
        phi = 1.0 / (1.0 + np.exp(-20.0 * (phi - 0.5)))
        
        hist["t"].append(t)
        hist["F"].append(F)

        t += dt
        step += 1
    

    label_str = f"a = {a_test:.2e}"

    if step >= max_steps and t < t_end:
        print(f"  [Warn] {label_str}: Reached max_steps={max_steps} before t_end={t_end}s (t={t:.3f}s).")
    sens_histories[label_str] = {"t": np.array(hist["t"]), "F": np.array(hist["F"])}

# 민감도 플롯
fig, ax = plt.subplots(figsize=(11, 6))
for label, hist in sens_histories.items():
    ax.plot(hist["t"], hist["F"]/1e3, label=label, linewidth=2.5)

ax.set_xlabel('Time [s]', fontsize=11)
ax.set_ylabel('Thrust [kN]', fontsize=11)
ax.legend(fontsize=10, title="Burn Rate Parameter")
ax.grid(True, alpha=0.3)
ax.set_title('Sensitivity: Thrust vs Burn Rate Coefficient (Cylindrical Grain)', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "sensitivity_burn_rate.png"), dpi=150, bbox_inches='tight')
plt.show()
print(f"[Save] sensitivity_burn_rate.png")

## 9. 요약 및 결론

In [ ]:
print("\n" + "="*80)
print(" " * 20 + "SIMULATION SUMMARY")
print("="*80)
print(f"\n[Config]")
print(f"  Grid: {N}³ = {N**3:,} points, dx={dx*1000:.2f}mm")
print(f"  Time: t_end={t_end}s, dt_base={dt_base*1000:.2f}ms (adaptive), dt_min={dt_min*1000:.3f}ms, max_steps={max_steps}")
print(f"  Physics: γ={gamma}, Tc={Tc}K, ρ_p={rho_p}kg/m³")
print(f"  Grain Count: {len(histories)}")

print(f"\n[Improvements]")
print(f"  1. ✓ FFT Laplacian acceleration")
print(f"  2. ✓ CFL-based adaptive timestep")
print(f"  3. ✓ Semi-implicit Allen-Cahn scheme")
print(f"  4. ✓ Enhanced visualization with markers")
print(f"  5. ✓ Performance metrics (Impulse, Impulse Density)")
print(f"  6. - 3D visualization (can add marching_cubes)")
print(f"  7. ✓ Sensitivity analysis (burn rate parameter)")

print(f"\n[Output Directory]")
print(f"  {OUT_DIR}/")
print(f"    - grain_comparison.png (4-panel overview)")
print(f"    - thrust_comparison.png (thrust curves)")
print(f"    - sensitivity_burn_rate.png (parameter study)")

print(f"\n[Next Steps for Further Development]")
print(f"  - Add 3D isosurface visualization (marching_cubes)")
print(f"  - Implement multi-phase grain (composite + aluminized)")
print(f"  - Neural network inverse design (NN-based grain optimization)")
print(f"  - Coupled CFD for erosive burning")
print(f"  - Export to CAD format (STL/STEP)")
print(f"  - Parametric DoE (design of experiments)")

print("\n" + "="*80)